In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import warnings
import pyxirr as irr
import nelson_siegel_svensson.calibrate as nss_cal
import nelson_siegel_svensson as nss
import nelson_siegel_svensson.calibrate as ns_cal
import nelson_siegel_svensson as ns
from matplotlib.pyplot import plot
warnings.filterwarnings('ignore')


In [2]:
# Funciones
def calcular_pd_acum(periodo, curva_pds):
    if periodo <= 8:
        pd_acum = 1 - (curva_pds.loc[np.floor(periodo), 'No_PD_acum'] * np.exp(curva_pds.loc[np.ceil(periodo), 'mu'] * (periodo - np.floor(periodo))))
    else:
        pd_acum = 1 -(curva_pds.loc[8, 'No_PD_acum'] * np.exp(curva_pds.loc[8, 'mu'] * (periodo - 8)))
    return pd_acum

In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import least_squares

In [4]:

# ==========
# 1) Cargar datos
# ==========
path = r"bin\Cartera_bonos 3.xlsx"  # ajustá si hace falta
bonds = pd.read_excel(path, sheet_name="Hoja 1")
cfs   = pd.read_excel(path, sheet_name="Hoja 5")
#cfs = cfs[~cfs['Ticker'].isin(['TC25P','T2X5','T5X4'])]

# Precio de mercado (usar Cotización; es el dirty price en tu tabla)
price = bonds.set_index("Ticker")["Cotización"].astype(float)

# Nos quedamos con los CF "depurados" (según tu hoja 2)
cfs = cfs.rename(columns={
    "Time to Payment": "t",
    "Stochastic Credit Risk Free Cashflow": "cf"})
cfs["t"]  = cfs["t"].astype(float)
cfs["cf"] = cfs["cf"].astype(float)


# ==========
# 2) Nelson–Siegel en tasa zero efectiva anual
#    z(t) = beta0 + beta1 * ((1-exp(-t/tau))/(t/tau)) + beta2 * (((1-exp(-t/tau))/(t/tau)) - exp(-t/tau))
# ==========
def ns_zero_rate(t, beta0, beta1, beta2, tau):
    t = np.asarray(t, dtype=float)
    x = np.maximum(t / tau, 1e-10)  # evitar división por cero
    a = (1 - np.exp(-x)) / x
    return beta0 + beta1 * a + beta2 * (a - np.exp(-x))

def discount_factor(t, params):
    beta0, beta1, beta2, tau = params
    z = ns_zero_rate(t, beta0, beta1, beta2, tau)
    # DF con capitalización efectiva anual:
    return (1.0 / (1.0 + z)) ** t

# ==========
# 3) Función de error (vector de errores por bono)
# ==========
tickers = sorted(cfs["Ticker"].unique())

def residuals(params):
    res = []
    for tk in tickers:
        cf_i = cfs.loc[cfs["Ticker"] == tk, ["t", "cf"]]
        t = cf_i["t"].to_numpy()
        cf = cf_i["cf"].to_numpy()
        pv = np.sum(cf * discount_factor(t, params))
        res.append(pv - float(price.loc[tk]))
    return np.array(res)
# ==========
# 4) Calibración
#    Inicializaciones razonables (ajustables):
# ==========
x0 = np.array([0.5, -0.5, 0.5, 1.0])  # beta0, beta1, beta2, tau

# Restricciones suaves: tau>0; para tasas reales CER pueden ser negativas,
# por eso NO acoto beta0/beta1/beta2 a ser >0.
lb = np.array([-2, -2, -2, 0.01])
ub = np.array([ 2.0,  2, 2.0, 50.0])

opt = least_squares(residuals, x0, bounds=(lb, ub))
params_hat = np.array([0.03507098, -1.00000000,  0.8390515,  0.76645203]) # opt.x
print("Params (beta0,beta1,beta2,tau):", params_hat)
print("RMSE precio:", np.sqrt(np.mean(opt.fun**2)))
# ==========
# 5) Curva zero en tus fechas objetivo (en años)
#    Vos pedís 30/6 de cada año. Si querés exactitud ACT/365 desde 28-Jun-24,
#    calculás t exacto con fechas; acá muestro nodos anuales aproximados:
# ==========
target_years = np.arange(1,22,1) # 1..21 años (2025..2045 aprox)
z_targets = ns_zero_rate(target_years, *params_hat)
df_targets = discount_factor(target_years, params_hat)

curve = pd.DataFrame({
    "t_years": target_years,
    "zero_rate_eff_annual": z_targets,
    "discount_factor": df_targets
})
print(curve)
#curve.to_excel(r"bin\curva_zero_ns_mal_final_3.xlsx", index=False)



ValueError: Worksheet named 'Hoja 5' not found

In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

# =========================
# 1) Cargar datos
# =========================
path = r"bin\Cartera_bonos 3.xlsx"
bonds = pd.read_excel(path, sheet_name="Hoja 1")
cfs   = pd.read_excel(path, sheet_name="Hoja 2")

price = bonds.set_index("Ticker")["Cotización"].astype(float)

cfs = cfs.rename(columns={"Time to Payment": "t"})
cfs["t"] = cfs["t"].astype(float)

COL_CONTRACTUAL = "Contractual Cashflow"
COL_DEPURADO    = "Stochastic Credit Risk Free Cashflow"

if COL_CONTRACTUAL not in cfs.columns:
    raise ValueError(f"Falta columna {COL_CONTRACTUAL} en Hoja 6.")
if COL_DEPURADO not in cfs.columns:
    raise ValueError(f"Falta columna {COL_DEPURADO} en Hoja 6.")

tickers = sorted(cfs["Ticker"].unique())

def build_bond_cfs(cf_col):
    out = {}
    for tk in tickers:
        sub = cfs.loc[cfs["Ticker"] == tk, ["t", cf_col]].dropna()
        t = sub["t"].to_numpy(float)
        cf = sub[cf_col].to_numpy(float)
        out[tk] = (t, cf)
    return out

bond_cfs_con = build_bond_cfs(COL_CONTRACTUAL)
bond_cfs_dep = build_bond_cfs(COL_DEPURADO)

# =========================
# 2) Nelson–Siegel (zero + DF continuo)
# =========================
def ns_zero_rate(t, beta0, beta1, beta2, tau):
    t = np.asarray(t, dtype=float)
    tau = float(tau)
    x = np.maximum(t / max(tau, 1e-8), 1e-12)
    a = (1 - np.exp(-x)) / x
    return beta0 + beta1 * a + beta2 * (a - np.exp(-x))

def df_cont(t, params):
    beta0, beta1, beta2, tau = params
    z = ns_zero_rate(t, beta0, beta1, beta2, tau)
    return np.exp(-z * np.asarray(t, dtype=float))

def pv_bond(t, cf, params):
    return float(np.sum(cf * df_cont(t, params)))

def fit_report(params, bond_cfs, label=""):
    rows = []
    for tk, (t, cf) in bond_cfs.items():
        if tk not in price.index:
            continue
        p_mkt = float(price.loc[tk])
        p_hat = pv_bond(t, cf, params)
        rel_err = (p_hat - p_mkt) / p_mkt
        rows.append((tk, p_mkt, p_hat, rel_err, np.max(t) if len(t) else np.nan))
    df = pd.DataFrame(rows, columns=["Ticker", "P_mkt", "P_hat", "rel_error", "max_t"])
    rmse_rel = float(np.sqrt(np.mean(df["rel_error"]**2))) if len(df) else np.nan
    print(f"\n[{label}] Params: {params}")
    print(f"[{label}] RMSE relativo precios: {rmse_rel:.6f}")
    print(df.sort_values("rel_error", key=lambda s: np.abs(s), ascending=False).head(10).to_string(index=False))
    return df

# =========================
# 3) Calibración: curva con crédito (contractual)
# =========================
def calibrate_ns_price(bond_cfs, x0=None):
    tk_list = [tk for tk in tickers if tk in price.index]

    def residuals(params):
        beta0, beta1, beta2, tau = params
        if tau <= 1e-6:
            return np.ones(len(tk_list)) * 1e3
        res = []
        for tk in tk_list:
            t, cf = bond_cfs[tk]
            p_mkt = float(price.loc[tk])
            p_hat = pv_bond(t, cf, params)
            res.append((p_hat - p_mkt) / p_mkt)
        return np.array(res, float)

    if x0 is None:
        x0 = np.array([0.08, -0.10, 0.05, 2.0])

    lb = np.array([-1, -2.00, -2.00, 0.01])
    ub = np.array([ 1,  2.00,  2.00, 50.0])

    opt = least_squares(residuals, x0, bounds=(lb, ub))
    return opt.x, opt

params_con, opt_con = calibrate_ns_price(bond_cfs_con)
fit_report(params_con, bond_cfs_con, label="CON (contractual)")

# =========================
# 4) Calibración: curva depurada con "no cruce" hasta LLP=4
# =========================
LLP = 4.0
t_ctrl = np.arange(0.5, LLP + 1e-9, 0.25)  # control fino en corto plazo

W_CROSS = 800.0   # con LLP corto, conviene hacerlo fuerte
W_DFMON = 500.0
NEG_FLOOR = -0.25
W_NEG = 0

# Penalización opcional: suavidad de forward (evitar dientes) post-LLP
W_FWD_SMOOTH = 30.0
t_fwd = np.arange(LLP, 30.0 + 1e-9, 1.0)  # forwards anuales desde LLP

def forward_rate(t, params, h=1e-4):
    # f(t) approx = -d ln DF / dt
    # ln DF(t) = -z(t)*t
    t = np.asarray(t, float)
    lnDF = np.log(df_cont(t, params))
    lnDF2 = np.log(df_cont(t + h, params))
    return -(lnDF2 - lnDF) / h

def calibrate_depured_no_cross(bond_cfs_dep, params_con):
    tk_list = [tk for tk in tickers if tk in price.index]

    def residuals(params):
        beta0, beta1, beta2, tau = params
        if tau <= 1e-6:
            return np.ones(len(tk_list) + 100) * 1e3

        res = []

        # (i) Fit a precios con CF depurados
        for tk in tk_list:
            t, cf = bond_cfs_dep[tk]
            p_mkt = float(price.loc[tk])
            p_hat = pv_bond(t, cf, params)
            res.append((p_hat - p_mkt) / p_mkt)

        # (ii) NO CRUCE solo hasta LLP
        z_dep = ns_zero_rate(t_ctrl, beta0, beta1, beta2, tau)
        z_con = ns_zero_rate(t_ctrl, *params_con)
        cross_viol = np.maximum(0.0, z_dep - z_con)
        res.extend(list(W_CROSS * cross_viol))

        # # (iii) DF decreciente
        # t_grid = np.linspace(0.25, 30.0, 160)
        # df_grid = df_cont(t_grid, params)
        # df_increase = np.maximum(0.0, np.diff(df_grid))
        # res.extend(list(W_DFMON * df_increase))

        # # (iv) Evitar negativos extremos (soft)
        # z_pen = ns_zero_rate(np.array([0.25, 0.5, 1, 2, 3, 4], float), beta0, beta1, beta2, tau)
        # neg_viol = np.maximum(0.0, (NEG_FLOOR - z_pen))
        # res.extend(list(W_NEG * neg_viol))

        # (v) Suavidad de forwards post-LLP (opcional)
        f = forward_rate(t_fwd, params)
        # penaliza cambios bruscos: segunda diferencia ~ curvatura
        f2 = np.diff(f, n=2)
        res.extend(list(W_FWD_SMOOTH * f2))

        return np.array(res, float)

    # Inicialización: cerca de la curva con crédito
    x0 = np.array(params_con, float).copy()
    x0[1] = x0[1] - 0.10  # permitir que baje el corto

    lb = np.array([-4, -4, -4, 0.05])
    ub = np.array([ 4,  4,  4, 30.0])

    opt = least_squares(residuals, x0, bounds=(lb, ub))
    return opt.x, opt

params_dep, opt_dep = calibrate_depured_no_cross(bond_cfs_dep, params_con)
fit_report(params_dep, bond_cfs_dep, label="DEP (depurado + no-cruce hasta 4y)")

# =========================
# 5) Verificación explícita del NO CRUCE en LLP
# =========================
t_check = np.arange(0.5, 10.5, 0.5)
z_con_chk = ns_zero_rate(t_check, *params_con)
z_dep_chk = ns_zero_rate(t_check, *params_dep)

mask = t_check <= LLP
cross_idx = np.where(z_dep_chk[mask] > z_con_chk[mask] + 1e-10)[0]
if len(cross_idx):
    print(f"\nWARNING: todavía cruza antes de LLP=4y. Primer cruce en t={t_check[mask][cross_idx[0]]} años.")
    print("Subí W_CROSS (p.ej. 800->1500) o revisá consistencia CF depurados vs precios.")
else:
    print("\nOK: no cruza (z_dep <= z_con) para t<=4y.")

# =========================
# 6) Curvas objetivo 1..21 años
# =========================
target_years = np.arange(1, 22, 1.0)
curve = pd.DataFrame({
    "t_years": target_years,
    "zero_con_credito": ns_zero_rate(target_years, *params_con),
    "zero_dep_no_cross": ns_zero_rate(target_years, *params_dep),
    "DF_con_credito": df_cont(target_years, params_con),
    "DF_dep_no_cross": df_cont(target_years, params_dep),
})
print("\nCurvas (targets 1..21):")
print(curve.to_string(index=False))
#curve.to_excel(r"bin\curva_zero_ns_mal.xlsx", index=False)



[CON (contractual)] Params: [ 0.09332786 -0.10041881  0.04224559  1.31492456]
[CON (contractual)] RMSE relativo precios: 0.011440
Ticker    P_mkt        P_hat  rel_error  max_t
 TZX27   170.00   165.544816  -0.026207   3.00
 TZX28   148.00   150.685978   0.018148   4.00
  T2X5   509.60   501.943012  -0.015025   0.63
 TZXD7   115.50   117.229591   0.014975   3.46
 TZXD5   136.75   138.281258   0.011197   1.46
 TZX25   158.70   160.281235   0.009964   1.00
  PARP 16340.00 16180.808918  -0.009742  14.50
 TZXD6   128.00   126.928319  -0.008373   2.46
  CUAP 20230.00 20330.570471   0.004971  21.50
 TC25P  4520.00  4540.921676   0.004629   0.82

[DEP (depurado + no-cruce hasta 4y)] Params: [ 0.04350374 -1.89602612  1.4387069   0.42902414]
[DEP (depurado + no-cruce hasta 4y)] RMSE relativo precios: 0.100816
Ticker   P_mkt        P_hat  rel_error  max_t
  DICP 31250.0 21971.649149  -0.296907   9.50
 TC25P  4520.0  3962.248131  -0.123396   0.82
 TZX25   158.7   176.370588   0.111346   1.00
  T